In [5]:
#write your names and students ids here

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import spacy
from gensim.models import Word2Vec, KeyedVectors


from scipy.sparse import dok_matrix
# Install spaCy (run in terminal/prompt)
import sys
#!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
#!{sys.executable} -m spacy download en



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Fabian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fabian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
from sklearn import preprocessing

In [20]:
data=pd.read_json ('News_Category_Dataset_v2.json', lines=True)

### Typical data analysis of a dataframe now

In [21]:
data.shape


(200853, 6)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category           200853 non-null  object        
 1   headline           200853 non-null  object        
 2   authors            200853 non-null  object        
 3   link               200853 non-null  object        
 4   short_description  200853 non-null  object        
 5   date               200853 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [23]:
#data.describe
data.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [24]:
print(data.groupby('category').size())

category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       4528
BUSINESS           5937
COLLEGE            1144
COMEDY             5175
CRIME              3405
CULTURE & ARTS     1030
DIVORCE            3426
EDUCATION          1004
ENTERTAINMENT     16058
ENVIRONMENT        1323
FIFTY              1401
FOOD & DRINK       6226
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
HOME & LIVING      4195
IMPACT             3459
LATINO VOICES      1129
MEDIA              2815
MONEY              1707
PARENTING          8677
PARENTS            3955
POLITICS          32739
QUEER VOICES       6314
RELIGION           2556
SCIENCE            2178
SPORTS             4884
STYLE              2254
STYLE & BEAUTY     9649
TASTE              2096
TECH               2082
THE WORLDPOST      3664
TRAVEL             9887
WEDDINGS           3651
WEIRD NEWS         2670
WELLNESS          17827
WOMEN              3490
WORLD NEWS         2177
WORLDPOST          2579
dtype: 

In [25]:
#we can see that many categories are similar.
#Let´s merge them. We will merge the more specific ones into the more general ones in case of doubt
#the code in the cell below was obtained from https://www.kaggle.com/rmisra/news-category-dataset/discussion/114275

In [26]:
data['category']=data['category'].replace({
"HEALTHY LIVING": "WELLNESS",
"QUEER VOICES": "GROUPS VOICES",
"BUSINESS": "BUSINESS & FINANCES",
"PARENTS": "PARENTING",
"BLACK VOICES": "GROUPS VOICES",
"THE WORLDPOST": "WORLD NEWS",
"STYLE": "STYLE & BEAUTY",
"GREEN": "ENVIRONMENT",
"TASTE": "FOOD & DRINK",
"WORLDPOST": "WORLD NEWS",
"SCIENCE": "SCIENCE & TECH",
"TECH": "SCIENCE & TECH",
"MONEY": "BUSINESS & FINANCES",
"ARTS": "ARTS & CULTURE",
"COLLEGE": "EDUCATION",
"LATINO VOICES": "GROUPS VOICES",
"CULTURE & ARTS": "ARTS & CULTURE",
"FIFTY": "MISCELLANEOUS",
"GOOD NEWS": "MISCELLANEOUS"
})

In [27]:
print(data.groupby('category').size())

category
ARTS & CULTURE          3878
BUSINESS & FINANCES     7644
COMEDY                  5175
CRIME                   3405
DIVORCE                 3426
EDUCATION               2148
ENTERTAINMENT          16058
ENVIRONMENT             3945
FOOD & DRINK            8322
GROUPS VOICES          11971
HOME & LIVING           4195
IMPACT                  3459
MEDIA                   2815
MISCELLANEOUS           2799
PARENTING              12632
POLITICS               32739
RELIGION                2556
SCIENCE & TECH          4260
SPORTS                  4884
STYLE & BEAUTY         11903
TRAVEL                  9887
WEDDINGS                3651
WEIRD NEWS              2670
WELLNESS               24521
WOMEN                   3490
WORLD NEWS              8420
dtype: int64


In [28]:
(data.groupby('category').size()).std()

7299.533828002569

As we may observe, the data is still highly imbalanced. Therefore further techniques will have to be applied to
make up for this

## Data cleaning:

We must  apply either lemmatization or stemming to the words.
We will try to go for lemmatization first as it depends on the context of the word.

We used the following link to guide ourselves https://www.machinelearningplus.com/nlp/lemmatization-examples-python/


//TODO: preproces words to remove apostrophes for example. isn´t -> is not

In [29]:
import spacy
from nltk.corpus import stopwords
spacy_lemmatiser = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
stop_words = stopwords.words('english')

#lemmatizer = WordNetLemmatizer()

def clean(text):

    # Define the sentence to be lemmatized
    text=text.lower() #making everything lowercase
    
    # Tokenize: Split the sentence into words
    #word_list = nltk.word_tokenize(text)
    #print(word_list)

    # Lemmatize list of words and join
    #lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    #print(lemmatized_output)
    
    # return lemmatized_output
    #return word_list
    
    # spacy Lemmatiser with POS tagging
    t_l_text = spacy_lemmatiser(text)
    lemmas = [token.lemma_ for token in t_l_text]
    
    lem_stop_text = [tok for tok in lemmas if tok not in stop_words]
            
    cleaned_text = " ".join(lem_stop_text) # Takes all the lemmas from text and joins 
    return cleaned_text



In [30]:

data['short_description'] = data['short_description'].apply(clean)
data['headline'] = data['headline'].apply(clean)


### ENCODING CLASSES

We need to transform strings into numerical labels. Remember, if we extend our model to work with NN, 
then we should transform to one-hot encoding instead

In [31]:
le = preprocessing.LabelEncoder()
le.fit(data['category'])
print(list(le.classes_))
print(le.transform(['FOOD & DRINK', 'GROUPS VOICES', 'HOME & LIVING']))


['ARTS & CULTURE', 'BUSINESS & FINANCES', 'COMEDY', 'CRIME', 'DIVORCE', 'EDUCATION', 'ENTERTAINMENT', 'ENVIRONMENT', 'FOOD & DRINK', 'GROUPS VOICES', 'HOME & LIVING', 'IMPACT', 'MEDIA', 'MISCELLANEOUS', 'PARENTING', 'POLITICS', 'RELIGION', 'SCIENCE & TECH', 'SPORTS', 'STYLE & BEAUTY', 'TRAVEL', 'WEDDINGS', 'WEIRD NEWS', 'WELLNESS', 'WOMEN', 'WORLD NEWS']
[ 8  9 10]


In [32]:
data['category encoded'] = le.transform(data['category'])
data.head()

,category,headline,authors,link,short_description,date,category encoded
0,CRIME,"2 mass shooting texas last week , 1 tv",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,leave husband . kill child . another day ameri...,2018-05-26,3
1,ENTERTAINMENT,smith join diplo nicky jam 2018 world cup 's o...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,course song .,2018-05-26,6
2,ENTERTAINMENT,hugh grant marrie first time age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,actor longtime girlfriend anna eberstein tie k...,2018-05-26,6
3,ENTERTAINMENT,jim carrey blast ' castrato ' adam schiff demo...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,actor give dem ass - kicking fight hard enough...,2018-05-26,6
4,ENTERTAINMENT,julianna margulie use donald trump poop bag pi...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,""" dietland "" actress say use bag "" really cath...",2018-05-26,6


### Splitting into train and test



In [19]:
train=data.sample(frac=0.8,random_state=200) #random state is a seed value
test=data.drop(train.index)

train.index= range(len(train))
test.reset_index= range(len(test))


### OUR MODELS

In [26]:
#the code from assignment 2 was changed here. We had a framework to start with this way.

class FeatureCreator:
    def __init__(self):
        self.voc = {}
        self.id = {}
        self.count = 0
    
    
    def createFeatureSet(self, data):
        # t1 and t2 are the 2 questions we want to compare
        for index, new in data.iterrows():
            
            for w in new['headline']:
                
                if w in self.voc:
                    self.voc[w] += 1
                else:
                    
                    self.voc[w] = 1
                    self.id[w] = self.count
                    self.count += 1;
                    
            for w in new['short_description']:
                if w in self.voc:
                    self.voc[w] += 1
                else:
                    self.voc[w] = 1
                    self.id[w] = self.count
                    self.count += 1;
                
        

class BowFeatureCreator(FeatureCreator): #runs in O(n^2)

    
     def createFeatures(self,data):
        
        id_len=len(self.id) #we will add this length to the index if we want to access the second question. It is made clear in the implementation

        features = dok_matrix((len(data),self.count*2)) #we will have twice the entries. One |VOC| x |2| one for the headline and the other one for the description
        print(features.shape)
        label = []
        i=0
        for i, row in data.iterrows():
            
            for w in row['headline']:
                if w in self.id:
                    features[i,self.id[w]] += 1
                    #features[i,self.id[w]+self.count] += 1
            for w in row['short_description']:
                if w in self.id:
                    features[i, self.id[w] + self.count]+= 1 #now there is one bow for the headline and another for the description
                    #features[i,self.id[w] + self.count] += 1
            print('i', i)
            label.append(row['category encoded'])
            
            
            #we want to make difference independent of the sentences,
        #that is, no negative values. We must take absolute value of features
        
        """
            # not the most efficient thing ever here, but it does the job
        i=0
        for j, row in data.iterrows():
            for w in row['headline']:
                if w in self.id:
                    if features[j,self.id[w]+self.count]<0:
                        features[j,self.id[w]+self.count] *= -1
            for w in row['short_description']:
                if w in self.id:
                     if features[j,self.id[w]+self.count]<0:
                        features[j,self.id[w] + self.count] *= -1
            i+=1.
            if i%1000 == 0:
                print('iteration: ', i)

        #features=self.absoluteValue(features,data)

        """
        
        
        return features,label

Calculate size of the vocabulary:

In [27]:
%%time
features=BowFeatureCreator()
features.createFeatureSet(train)


Wall time: 13 s


In [28]:
print(len(features.id))
print(len(features.voc))
print(features.count)

#print(train.shape)
print(len(train))


for k,v in features.id.items():
    if v > len(features.id):
        print(k)
        print(v)

498
498
498
160682


In [29]:
f,l = features.createFeatures(train)
print(features.count)

(160682, 997)
iteration:  1000.0
iteration:  2000.0
iteration:  3000.0
iteration:  4000.0
iteration:  5000.0
iteration:  6000.0
iteration:  7000.0
iteration:  8000.0
iteration:  9000.0
iteration:  10000.0
iteration:  11000.0
iteration:  12000.0
iteration:  13000.0
iteration:  14000.0
iteration:  15000.0
iteration:  16000.0
iteration:  17000.0
iteration:  18000.0
iteration:  19000.0
iteration:  20000.0
iteration:  21000.0
iteration:  22000.0
iteration:  23000.0
iteration:  24000.0
iteration:  25000.0
iteration:  26000.0
iteration:  27000.0
iteration:  28000.0
iteration:  29000.0
iteration:  30000.0
iteration:  31000.0
iteration:  32000.0
iteration:  33000.0
iteration:  34000.0
iteration:  35000.0
iteration:  36000.0
iteration:  37000.0
iteration:  38000.0
iteration:  39000.0
iteration:  40000.0
iteration:  41000.0
iteration:  42000.0
iteration:  43000.0
iteration:  44000.0
iteration:  45000.0
iteration:  46000.0
iteration:  47000.0
iteration:  48000.0
iteration:  49000.0
iteration:  500

### Multinomial Naive Bayes Classifier

In [30]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(f, l)

MultinomialNB()

### MODEL EVALUATION

We need to create many more variations with many different features

In [31]:
from sklearn.metrics import classification_report
testf,testl = features.createFeatures(test)
predicted_nb=mnb.predict(testf)
print(metrics.classification_report(testl, predicted_nb));

(40171, 997)


IndexError: row index (40173) out of range

In [ ]:
.
.
.
.
.
..
.
.
.

.
.
.
.
..
.

.
.
.
.
.
.
.
..

.
.
.Ignore from here onwards

# IGNORE FROM HERE ONWARDS

//we will trin the model once we are done with the cleaning

First we split the data such that the proportion of category labels is the same in train and test. Remember, category will be our target

In [ ]:
train, test = train_test_split(data, test_size=0.2, stratify=data['category'])

Leave that for later, do not pay too much attention for now

In [ ]:

class_weights = dict(zip(np.unique(y_train), class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)))

Then you use the class weights during the training process:

train_history = model.fit( train_dataset, steps_per_epoch=n_steps, class_weight=class_weights )

In [ ]:
data.loc[1,'headline']

In [ ]:
sentences=[]
for i, row in data.iterrows():
    sentences.append(row['headline'])

model_embeddings=Word2Vec(vector_size=20, min_count=1)
model_embeddings.build_vocab(sentences, progress_per=10000)
model_embeddings.train(sentences, total_examples=model_embeddings.corpus_count, epochs=30,report_delay=1)
#model_embeddings.build(data['headlines'])


In [ ]:
model_embeddings.wv.most_similar(positive=["korea"])
model_embeddings.wv.most_similar(positive=["Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song"])